# Problem 1

* Current Stock Price 151.03

* Strike Price 165

* Current Date 03/13/2022

* Options Expiration Date 04/15/2022

* Risk Free Rate of 4.25%

* Continuously Compounding Coupon of 0.53%

Implement the closed form Greeks for GBSM. Implement a finite difference derivative calculation.

Compare the values between the two methods for both a call and a put.

Implement the binomial tree valuation for American options with and without discrete dividends. Assume the stock above:

* Pays dividend on 4/11/2022 of $0.88

Calculate the value of the call and the put. Calculate the Greeks of each.

What is the sensitivity of the put and call to a change in the dividend amount?

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.stats import norm

In [2]:
S = 151.03
X = 165
r = 0.0425
coupon = 0.0053
b = r - coupon
sigma = 0.2
current_date = datetime(2022,3,13)
exp_date = datetime(2022,4,15)

In [3]:
def time_to_maturity(current_date, exp_date):
    ttm = (exp_date - current_date).days / 365
    return ttm

def cal_d1_d2(S, X, T, sigma, b):
    d1 = (np.log(S/X)+(b + (sigma**2)/2)*T)/(sigma* np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return d1, d2
T = time_to_maturity(current_date, exp_date)

In [4]:
def delta_gbsm(option_type, S, X, T, sigma, b, r):
    d1, d2 = cal_d1_d2(S, X, T, sigma, b)
    if option_type == "Call":
        delta = np.exp((b-r)*T) * norm.cdf(d1)
    else:
        delta = np.exp((b-r)*T) * (norm.cdf(d1)-1)
    return delta

In [5]:
print('For the closed form greeks for GBSM:')
print("Call's Delta is ", delta_gbsm('Call', S, X, T, sigma, b, r))
print("Put's Delta is ", delta_gbsm('Put', S, X, T, sigma, b, r))

For the closed form greeks for GBSM:
Call's Delta is  0.08297130333914773
Put's Delta is  -0.9165496333661425


In [6]:
def gamma_gbsm(S, X, T, sigma, b, r):
    d1, d2 = cal_d1_d2(S, X, T, sigma, b)
    gamma = (norm.pdf(d1)*np.exp((b-r)*T)) / (S * sigma *np.sqrt(T))
    return gamma

print("Call's Gamma is ", gamma_gbsm( S, X, T, sigma, b, r))
print("Put's Gamma is ", gamma_gbsm( S, X, T, sigma, b, r))

Call's Gamma is  0.016822916101852648
Put's Gamma is  0.016822916101852648


In [7]:
def vega_gbsm(S, X, T, sigma, b, r):
    d1, d2 = cal_d1_d2(S, X, T, sigma, b)
    vega = S * np.exp((b-r)*T) * norm.pdf(d1) * np.sqrt(T)
    return vega
print("Call's Vega is ", vega_gbsm( S, X, T, sigma, b, r))
print("Put's Vega is ", vega_gbsm( S, X, T, sigma, b, r))

Call's Vega is  6.938710929513443
Put's Vega is  6.938710929513443


In [8]:
def theta_gbsm(option_type, S, X, T, sigma, b, r):
    d1, d2 = cal_d1_d2(S, X, T, sigma, b)
    if option_type == "Call":
        theta = -(S*np.exp((b-r)*T)*norm.pdf(d1)*sigma)/(2*np.sqrt(T))\
                -(b-r)*S*np.exp((b-r)*T)*norm.cdf(d1)\
                -r*X*np.exp(-r*T)*norm.cdf(d2)
    else:
        theta = -(S*np.exp((b-r)*T)*norm.pdf(d1)*sigma)/(2*np.sqrt(T))\
                +(b-r)*S*np.exp((b-r)*T)*norm.cdf(-d1)\
                +r*X*np.exp(-r*T)*norm.cdf(-d2)
    return theta
print("Call's Theta is ", theta_gbsm( "Call", S, X, T, sigma, b, r))
print("Put's Theta is ", theta_gbsm( "Put", S, X, T, sigma, b, r))

Call's Theta is  -8.126522359668838
Put's Theta is  -1.9409914783019566


In [9]:
def rho_gbsm(option_type, S, X, T, sigma, b, r):
    d1, d2 = cal_d1_d2(S, X, T, sigma, b)
    if option_type == "Call":
        rho = T*X*np.exp(-r*T)*norm.cdf(d2)
    else:
        rho = -T*X*np.exp(-r*T)*norm.cdf(-d2)
    return rho
print("Call's Rho is ", rho_gbsm( "Call", S, X, T, sigma, b, r))
print("Put's Rho is ", rho_gbsm( "Put", S, X, T, sigma, b, r))

Call's Rho is  1.1025939156368187
Put's Rho is  -13.758003122735788


In [10]:
def carry_rho_gbsm(option_type, S, X, T, sigma, b, r):
    d1, d2 = cal_d1_d2(S, X, T, sigma, b)
    if option_type == "Call":
        carry_rho = T*S*np.exp((b-r)*T)*norm.cdf(d1)
    else:
        carry_rho = -T*S*np.exp((b-r)*T)*norm.cdf(-d1)
    return carry_rho
print("Call's Carry Rho is ", carry_rho_gbsm( "Call", S, X, T, sigma, b, r))
print("Put's Carry Rho is ", carry_rho_gbsm( "Put", S, X, T, sigma, b, r))
# closed form calculation of the Greeks can only be applied for GBSM, if we use binomial trees to price options, we can only use finite difference calculation

Call's Carry Rho is  1.132953825011723
Put's Carry Rho is  -12.515271800549371


In [11]:
import inspect

# Implement a finite difference derivative calculation
def first_order_derivative(function, x, delta):
    result = (function(x+delta) - function(x-delta)) / (2*delta)
    return result

def second_order_devirvative(function, x, delta):
    result = (function(x+delta) + function(x-delta) - 2*function(x)) / (delta**2)
    return result

In [12]:
def cal_derivative_wrt_one(function, order, object_arg, delta = 1e-3):
    all_args = list(inspect.signature(function).parameters.keys())
    orders_dic = {1:first_order_derivative, 2:second_order_devirvative}

    def cal_derivative(*args, **kwargs):
        args_dic = dict(list(zip(all_args, args)) + list(kwargs.items()))
        value_arg = args_dic.pop(object_arg)

        def trans_into_one_arg(x):
            all_args = {object_arg:x, **args_dic}
            return function(**all_args)
        return orders_dic[order](trans_into_one_arg, value_arg, delta)
    return cal_derivative

In [13]:
def gbsm(option_type, S, X, T, sigma, r, b):
    d1 = (np.log(S/X)+(b + (sigma**2)/2)*T)/(sigma* np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "Call":
        call_value = S * np.exp((b-r)*T)*norm.cdf(d1) - X * np.exp(-r*T)*norm.cdf(d2)
        return call_value
    else:
        put_value = X * np.exp(-r*T)*norm.cdf(-d2) - S * np.exp((b-r)*T)*norm.cdf(-d1)
        return put_value

In [14]:
print("For finite difference:")
gbsm_delta = cal_derivative_wrt_one(gbsm, 1, 'S')
# 1 means the derivative we are taking is 1, the first-order derivative measures the rate of change of the function with respect to one of its variables
# "S" means the derivative we are taking is against "S", the underlying stock price, because delta measures how much the price of an option is expected to move for a $1 change in the price of the underlying asset
print("Call's Delta is ", gbsm_delta( "Call", S, X, T, sigma, r, b))
print("Put's Delta is ", gbsm_delta( "Put", S, X, T, sigma, r, b))
gbsm_gamma = cal_derivative_wrt_one(gbsm, 2 ,'S')
# 2 means the derivative we are taking is 2, the second-order derivative measures the rate of change of the rate of change (i.e., the curvature or the acceleration) of the function with respect to one of its variables
# "S" means the derivative we are taking is against "S", the underlying stock price, because Gamma assesses how much Delta will change for a $1 change in the stock price.
print("Call's Gamma is ", gbsm_gamma( "Call", S, X, T, sigma, r, b))
print("Put's Gamma is ", gbsm_gamma( "Put", S, X, T, sigma, r, b))
gbsm_vega = cal_derivative_wrt_one(gbsm, 1 ,'sigma')
# "sigma" means the derivative we are taking is against "sigma", the volatility of the underlying asset, because Vega measures the sensitivity of the option price to changes in the volatility of the underlying asset
print("Call's Vega is ", gbsm_vega( "Call", S, X, T, sigma, r, b))
print("Put's Vega is ", gbsm_vega( "Put", S, X, T, sigma, r, b))
gbsm_theta = cal_derivative_wrt_one(gbsm, 1 ,'T')
# "T" means the derivative we are taking is against "T", the time, because Theta quantifies the rate at which an option's value declines as the expiration date approaches.
print("Call's Theta is ", -gbsm_theta( "Call", S, X, T, sigma, r, b))
print("Put's Theta is ", -gbsm_theta( "Put", S, X, T, sigma, r, b))
gbsm_rho = cal_derivative_wrt_one(gbsm, 1 ,'r')
# "-": while the raw computation yields a positive value (the rate at which the option value decreases per time unit), you negate it to express Theta in its conventional form, which shows the loss in option value over time from the holder's perspective.
# "r" means the derivative we are taking is against "r", the risk-free interest rate, because Rho assesses the sensitivity of the option price to changes in the risk-free interest rate.
print("Call's Rho is ", gbsm_rho( "Call", S, X, T, sigma, r, b))
print("Put's Rho is ", gbsm_rho( "Put", S, X, T, sigma, r, b))
gbsm_carry_rho = cal_derivative_wrt_one(gbsm, 1 ,'b')
# "b" means the derivative we are taking is against "b", the cost of carry rate, because b captures the impact of both interest rates and dividend yields (if any) on the option's value.
print("Call's Carry Rho is ", gbsm_carry_rho( "Call", S, X, T, sigma, r, b))
print("Put's Carry Rho is ", gbsm_carry_rho( "Put", S, X, T, sigma, r, b))

For finite difference:
Call's Delta is  0.08297130374668171
Put's Delta is  -0.9165496329472944
Call's Gamma is  0.016822911064195978
Put's Gamma is  0.016822951920403284
Call's Vega is  6.938653056250743
Put's Vega is  6.93865305626673
Call's Theta is  -8.126308803761084
Put's Theta is  -1.9407779203106656
Call's Rho is  -0.030359909416688424
Put's Rho is  -1.2427313238703164
Call's Carry Rho is  1.1329550097096686
Put's Carry Rho is  -12.515270634423814


In [15]:
# No dividend binomial tree
def bt_no_div(call, underlying, strike, ttm, rf, b, ivol, N):
    # the type of option (call),
    # the current price of the underlying asset (underlying),
    # the strike price of the option (strike),
    # time to maturity (ttm),
    # risk-free interest rate (rf),
    # cost-of-carry rate (b),
    # implied volatility (ivol),
    # the number of time steps (N)
    dt = ttm/N
    u = np.exp(ivol*np.sqrt(dt))
    d = 1/u
    pu = (np.exp(b*dt)-d)/(u-d)
    pd = 1.0-pu
    df = np.exp(-rf*dt)
    z = 1 if call else -1

    def nNodeFunc(n):
        return (n+1)*(n+2) // 2
    def idxFunc(i,j):
        return nNodeFunc(j-1)+i
    nNodes = nNodeFunc(N)

    optionValues = [0.0] * nNodes

    for j in range(N,-1,-1):
        for i in range(j,-1,-1):
            idx = idxFunc(i,j)
            price = underlying*u**i*d**(j-i)
            optionValues[idx] = max(0,z*(price-strike))

            if j < N:
                optionValues[idx] = max(optionValues[idx], df*(pu*optionValues[idxFunc(i+1,j+1)] + pd*optionValues[idxFunc(i,j+1)]))

    return optionValues[0]

In [16]:
from typing import List

def bt_with_div(call: bool, underlying: float, strike: float, ttm: float, rf: float, b:float, divAmts: List[float], divTimes: List[int], ivol: float, N: int):
    # Actually b = rf in discrete dividend condition, because the dividend component of the cost-of-carry is already accounted for in the adjustments to the underlying asset's price.
    # if there are no dividends or the first dividend is outside out grid, then calculation would be the same as bt_no_div()

    # call: The type of option, where True indicates a call option and False indicates a put option.
    # underlying: The current price of the underlying asset (such as a stock) for which the option is written.
    # strike: The strike price of the option, which is the fixed price at which the holder can buy (for a call) or sell (for a put) the underlying asset.
    # ttm: Time to maturity of the option, expressed in years. It represents how much time is left until the option expires.
    # rf: The risk-free interest rate, typically representing the theoretical return of an investment with no risk of financial loss.
    # b: The cost-of-carry rate, which in the context of this function, is assumed to be equal to the risk-free interest rate (rf). This simplification is due to the discrete treatment of dividends.
    # divAmts: A list of dividend amounts, with each element in the list representing the amount of a single dividend payment.
    # divTimes: A list of times at which dividends are paid, expressed as the number of time steps (from the total N steps) until each dividend payment.
    # ivol: Implied volatility of the underlying asset, reflecting the market's expectation of the asset's future volatility over the life of the option.
    # N: The number of time steps used in the binomial tree model, affecting the granularity and accuracy of the option price calculation.

    if not divAmts or not divTimes or divTimes[0] > N:
        return bt_no_div(call, underlying, strike, ttm, rf, b, ivol, N)

    dt = ttm / N
    u = np.exp(ivol * np.sqrt(dt))
    d = 1 / u
    pu = (np.exp(b * dt) - d) / (u - d)
    pd = 1 - pu
    df = np.exp(-rf * dt)
    z = 1 if call else -1

    def nNodeFunc(n: int) -> int:
        return int((n + 1) * (n + 2) / 2)

    def idxFunc(i: int, j: int) -> int:
        return nNodeFunc(j - 1) + i

    nDiv = len(divTimes)
    nNodes = nNodeFunc(divTimes[0])

    optionValues = [0] * nNodes

    for j in range(divTimes[0], -1, -1):
        for i in range(j, -1, -1):
            idx = idxFunc(i, j)
            price = underlying * (u ** i) * (d ** (j - i))

            if j < divTimes[0]:
                #times before the dividend working backward induction
                optionValues[idx] = max(0, z * (price - strike))
                optionValues[idx] = max(optionValues[idx], df * (pu * optionValues[idxFunc(i + 1, j + 1)] + pd * optionValues[idxFunc(i, j + 1)]))
            else:
                #time of the dividend
                valNoExercise = bt_with_div(call, price - divAmts[0], strike, ttm - divTimes[0] * dt, rf, b, divAmts[1:], [t - divTimes[0] for t in divTimes[1:]], ivol, N - divTimes[0])
                valExercise = max(0, z * (price - strike))
                optionValues[idx] = max(valNoExercise, valExercise)

    return optionValues[0]

In [17]:
N = 200
div = [0.88]
div_date = datetime(2022, 4, 11)
div_time = [round((div_date - current_date).days / (exp_date - current_date).days * N)]

In [18]:
print("Using binomial tree")
print("For the condition without dividend:")
print("The value of call option is ", bt_no_div(True, S, X, T, r, b, sigma, N))
print("The value of put option is ", bt_no_div(False, S, X, T, r, b, sigma, N))
b = 0.0425
print("For the condition with dividend:")
print("The value of call option is ", bt_with_div(True, S, X, T, r, b, div, div_time, sigma, N))
print("The value of put option is ", bt_with_div(False, S, X, T, r, b, div, div_time, sigma, N))

Using binomial tree
For the condition without dividend:
The value of call option is  0.33600393488438995
The value of put option is  14.036960189311927
For the condition with dividend:
The value of call option is  0.2986182507141458
The value of put option is  14.556578296240984


In [19]:
print("Using binomial tree, calculate the greeks:")
bt_delta = cal_derivative_wrt_one(bt_with_div, 1, 'underlying')
# 1 means the derivative we are taking is 1, the first-order derivative measures the rate of change of the function with respect to one of its variables
# "underlying" means the derivative we are taking is against "underlying", the underlying stock price, because delta measures how much the price of an option is expected to move for a $1 change in the price of the underlying asset: note the actual variable that we will put into cal_derivative_wrt_one varies with variable we want to derive against in the target function, so in gbsm() it is "S", in bt_with_div it is "underlying"
print("Call's Delta is ", bt_delta(True, S, X, T, r, b, div, div_time, sigma, N))
print("Put's Delta is ", bt_delta(False, S, X, T, r, b, div, div_time, sigma, N))

bt_gamma = cal_derivative_wrt_one(bt_with_div, 2 ,'underlying')
print("Call's Gamma is ", bt_gamma( True, S, X, T, r, b, div, div_time, sigma, N))
print("Put's Gamma is ", bt_gamma( False, S, X, T, r, b, div, div_time, sigma, N))

bt_vega = cal_derivative_wrt_one(bt_with_div, 1 ,'ivol')
print("Call's Vega is ", bt_vega( True, S, X, T, r, b, div, div_time, sigma, N))
print("Put's Vega is ", bt_vega( False, S, X, T, r, b, div, div_time, sigma, N))

bt_theta = cal_derivative_wrt_one(bt_with_div, 1 ,'ttm')
print("Call's Theta is ", -bt_theta( True, S, X, T, r, b, div, div_time, sigma, N))
print("Put's Theta is ", -bt_theta( False, S, X, T, r, b, div, div_time, sigma, N))

bt_rho = cal_derivative_wrt_one(bt_with_div, 1 ,'rf')
print("Call's Rho is ", bt_rho( True, S, X, T, r, b, div, div_time, sigma, N))
print("Put's Rho is ", bt_rho( False, S, X, T, r, b, div, div_time, sigma, N))

bt_carry_rho = cal_derivative_wrt_one(bt_with_div, 1 ,'b')
print("Call's Carry Rho is ", bt_carry_rho( True, S, X, T, r, b, div, div_time, sigma, N))
print("Put's Carry Rho is ", bt_carry_rho( False, S, X, T, r, b, div, div_time, sigma, N))

Using binomial tree, calculate the greeks:
Call's Delta is  0.07257286328979373
Put's Delta is  -0.9383141177661258
Call's Gamma is  -8.881784197001252e-10
Put's Gamma is  1.3287149158713873e-05
Call's Vega is  6.319443776511474
Put's Vega is  5.675482057655223
Call's Theta is  -7.467912305700292
Put's Theta is  -0.4489716873523619
Call's Rho is  -0.02437109966921258
Put's Rho is  -1.160866467771804
Call's Carry Rho is  0.9626647836451507
Put's Carry Rho is  -11.311095439340946


In [20]:
# Sensitivity of the put and call to a change in dividend amount
delta = 1e-3
div_up = [0.88 + delta]
div_down = [0.88 - delta]
call_up = bt_with_div(True, S, X, T, r, b, div_up, div_time, sigma, N)
call_down = bt_with_div(True, S, X, T, r, b, div_down, div_time, sigma, N)
call_sens_to_div_amount = (call_up - call_down) / (2*delta)

put_up = bt_with_div(False, S, X, T, r, b, div_up, div_time, sigma, N)
put_down = bt_with_div(False, S, X, T, r, b, div_down, div_time, sigma, N)
put_sens_to_div_amount = (put_up - put_down) / (2*delta)
print(f"Sensitivity to dividend amount: Call: {call_sens_to_div_amount:.3f}, Put: {put_sens_to_div_amount:.3f}")

Sensitivity to dividend amount: Call: -0.021, Put: 0.941


# Problem 2

Using the options portfolios from Problem3 last week (named problem2.csv in this week’s repo) and assuming :

* American Options

* Current Date 03/03/2023

* Current AAPL price is 165

* Risk Free Rate of 4.25%

* Dividend Payment of 1.00 on 3/15/2023

Using DailyPrices.csv. Fit a Normal distribution to AAPL returns – assume 0 mean return. Simulate AAPL returns 10 days ahead and apply those returns to the current AAPL price (above). Calculate Mean, VaR and ES.

Calculate VaR and ES using Delta-Normal.

Present all VaR and ES values a dollar loss, not percentages.

Compare these results to last week’s results.

In [21]:
from scipy.optimize import fsolve

N = 30
portfolios = pd.read_csv("problem2.csv", parse_dates=["ExpirationDate"]) # parse these as dates

current_date = datetime(2023,3,3)
S_aapl = 151.03

def find_iv(call, underlying, strike, ttm, rf, b, divAmts, divTimes, N, price, guess=0.5):
    def f(ivol):
        return bt_with_div(call, underlying, strike, ttm, rf, b, divAmts, divTimes, ivol, N) - price
    return fsolve(f, guess)[0]

In [22]:
ivs = []
for i in range(len(portfolios.index)):
    if portfolios["Type"][i] == 'Option':
        if portfolios["OptionType"][i] == "Call":
            call = True
        elif portfolios["OptionType"][i] =="Put":
            call = False
        X = portfolios["Strike"][i]
        T = time_to_maturity(current_date, portfolios["ExpirationDate"][i])
        N = 100
        div = [1]
        div_date = datetime(2023, 3, 15)
        div_time = [int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)]
        price = portfolios["CurrentPrice"][i]
        sigma = find_iv(call, S_aapl, X, T, r, b, div, div_time, N, price)
        ivs.append(sigma)
    else:
        ivs.append(0)
portfolios['IV'] = ivs

In [23]:
def cal_portfolio_value(portfolios, underlying_price, current_date):
    portfolio_values = pd.DataFrame(index=portfolios.index)
    portfolio_values['Portfolio'] = portfolios['Portfolio']

    one_values = []
    for i in range(len(portfolios.index)):
        if portfolios['Type'][i] == "Stock":
            one_p = underlying_price
        else:
            if portfolios["OptionType"][i] == "Call":
                call = True
            elif portfolios["OptionType"][i] =="Put":
                call = False
            S = underlying_price
            X = portfolios["Strike"][i]
            T = time_to_maturity(current_date, portfolios["ExpirationDate"][i])
            iv = portfolios['IV'][i]
            div = [1]
            div_date = datetime(2023, 3, 15)
            div_time = [int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)]
            one_p = bt_with_div(call, S, X, T, r, b, div, div_time, iv, N)
        one_values.append(one_p)
    portfolio_values['Value'] = portfolios["Holding"] * np.array(one_values)
    return portfolio_values.groupby('Portfolio').sum()

portfolio_values_diff = cal_portfolio_value(portfolios, S_aapl, current_date)

In [24]:
from lib_hcy_week5 import myfunctions as mf
import scipy

prices = pd.read_csv('DailyPrices.csv')
all_returns = mf.return_calculate(prices, method="LOG")
AAPL_returns = all_returns['AAPL']
AAPL_ret = AAPL_returns - AAPL_returns.mean()

/Users/yanghaochen/opt/anaconda3/lib/python3.9/site-packages/lib_hcy_week5/myfunctions.py:243: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars[i]] = p2[:,i]


In [25]:
np.random.seed(123)
mu, std = norm.fit(AAPL_ret)
sim_returns = scipy.stats.norm(mu, std).rvs((10, 100))
sim_prices = S_aapl * np.exp(sim_returns.sum(axis=0)) # sum by each column, get 100 different simulated 10-day-ahead price

portfolio_values_sim = cal_portfolio_value(portfolios, S_aapl, current_date)

In [26]:
for i in sim_prices:
    temp_one_pv = cal_portfolio_value(portfolios, i, current_date)
    portfolio_values_sim[str(i)] = temp_one_pv['Value']

/var/folders/zd/m53hqz696cnd23z0k2c1g9zc0000gn/T/ipykernel_25192/2071873280.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_values_sim[str(i)] = temp_one_pv['Value']


In [27]:
portfolio_values_sim.drop('Value', axis=1, inplace=True)
portfolios["CurrentValue"] = portfolios["CurrentPrice"] * portfolios["Holding"]
portfolio_values_curr = portfolios.groupby('Portfolio')['CurrentValue'].sum()
sim_value_changes = (portfolio_values_sim.T - portfolio_values_curr).T
print(sim_value_changes)

              168.61117442122656  131.4808955639731  167.23508102830814  \
Portfolio                                                                 
Call                   12.880666          -6.050472           11.677396   
CallSpread              3.940920          -3.906412            3.741479   
CoveredCall             6.475374         -15.731407            6.202968   
ProtectedPut           14.877766          -7.671075           13.575867   
Put                    -4.256611          14.599312           -4.132419   
PutSpread              -2.583390           5.517591           -2.500519   
Stock                  17.581174         -19.549104           16.205081   
Straddle                8.624055           8.548840            7.544977   
SynLong                17.137277         -20.649784           15.809815   

              157.71778910127182  150.3900773508654  177.96666455091648  \
Portfolio                                                                 
Call                    

In [28]:
def cal_var(sim_data, alpha = 0.05):
    sim_data_sorted = np.sort(sim_data)
    var = sim_data_sorted[int(alpha * len(sim_data))] * (-1)
    return var

def cal_es(sim_data, alpha = 0.05):
    sim_data_sorted = np.sort(sim_data)
    var = sim_data_sorted[int(alpha * len(sim_data))] * (-1)
    return -np.mean(sim_data_sorted[sim_data_sorted <= -var])

In [29]:
# Calculate the Mean, VaR and ES, and print the results
result = pd.DataFrame(index=sim_value_changes.index)
result['Mean'] = sim_value_changes.mean(axis=1) # calculate mean by each row
result['VaR'] = sim_value_changes.apply(lambda x:cal_var(x), axis=1)
result['ES'] = sim_value_changes.apply(lambda x:cal_es(x), axis=1)
print(result)

                  Mean        VaR         ES
Portfolio                                   
Call          1.073965   6.000911   6.230463
CallSpread   -0.148355   3.862773   4.067639
CoveredCall  -2.121938  15.328753  18.253996
ProtectedPut  1.048025   7.613934   7.833054
Put           2.263459   4.256611   4.547216
PutSpread     0.754630   2.583390   2.789961
Stock        -0.882675  19.122870  22.132856
Straddle      3.337425   0.003846   0.004879
SynLong      -1.189494  20.195068  23.372346


In [30]:
#Delta Normal

current_date = datetime(2023, 3, 3)
div_date = datetime(2023, 3, 15)
r = 0.0425
div = [1]

cal_amr_delta_num = cal_derivative_wrt_one(bt_with_div, 1, 'underlying') # function to calculate delta with binomial trees and with discrete dividend

deltas = []
for i in range(len(portfolios.index)):
  if portfolios["Type"][i] == "Stock":
    deltas.append(1)
  else:
    if portfolios["OptionType"][i] == "Call":
        call = True
    elif portfolios["OptionType"][i] =="Put":
        call = False
    ivol = portfolios["IV"][i]
    X = portfolios["Strike"][i]
    T = ((portfolios["ExpirationDate"][i] - current_date).days - 10) / 365
    div_time = [int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)]
    delta = cal_amr_delta_num(call, S_aapl, X, T, r, b, div, div_time, ivol, N)
    deltas.append(delta) # get a list of deltas corresponding to all asset (rows) in portfolios.csv

In [31]:
# Store the deltas in portfolios
portfolios["deltas"] = deltas

portfolio_deltas = pd.DataFrame(index=portfolios.index)
portfolio_deltas['Portfolio'] = portfolios['Portfolio']

portfolio_deltas['Delta'] = portfolios["Holding"] * portfolios["deltas"]
portfolio_delta = portfolio_deltas.groupby('Portfolio').sum()
# portfolio_delta is a new dataframe created, in which there is asset names, and asset delta compared to stock

prices_df = pd.DataFrame(np.tile(sim_prices, (len(portfolio_delta), 1)),
                         index=portfolio_delta.index,
                         columns=[str(i) for i in sim_prices])
# create a dataframe, in which row index == portfolio_delta, asset names; column index == simulated prices of AAPL; each value in the block == the value in the first block of that column, which are all stock price, making it easier for later delta modification

delta_prices = portfolio_delta['Delta'].values[:, np.newaxis] * prices_df
# element wise multiplication, use each delta to scale each row

hedge_value = portfolio_values_sim.sub(delta_prices, fill_value=0)
# sub is done for each block of the same position
current_stock = portfolio_delta * S_aapl
current_stock = current_stock.rename(columns={'Delta': 'CurrentValue'})

current_pfl = pd.DataFrame(portfolio_values_curr)

current_hedge = pd.DataFrame(current_pfl).sub(current_stock)

hedge_value_changes = (hedge_value.T - current_hedge['CurrentValue']).T
# transpose is used to allow same current value be subtracted from the same column of hedge_value.T, which is each portfolio

result = pd.DataFrame(index=hedge_value_changes.index)
result['Mean'] = hedge_value_changes.mean(axis=1)
result['VaR'] = hedge_value_changes.apply(lambda x:cal_var(x), axis=1)
result['ES'] = hedge_value_changes.apply(lambda x:cal_es(x), axis=1)
print(result)

                  Mean       VaR        ES
Portfolio                                 
Call          1.567368 -0.020529 -0.005258
CallSpread    0.104107  1.087634  2.002503
CoveredCall  -1.609392  5.839371  7.286841
ProtectedPut  1.681277  0.018343  0.021242
Put           1.859991 -0.023125 -0.005794
PutSpread     0.539901 -0.006245  0.070871
Stock         0.000000 -0.000000 -0.000000
Straddle      3.427359 -0.046552 -0.011534
SynLong      -0.292622  0.927251  1.042719


VaR and ES from Normal Distribution Simulation:
This method calculates VaR and ES based on the simulated future prices of the underlying asset (AAPL) and the resultant portfolio values.
It captures the non-linear behavior of the options in the portfolio and the full range of potential market scenarios as reflected in the simulated prices.
This approach provides a more comprehensive view of the risk, considering the actual behavior of the options under various price scenarios.

VaR and ES from Delta-Normal Method:
After the initial simulation of future prices, this method adjusts the simulated portfolio values based on the deltas of the options. Delta, a measure of the option's price sensitivity to the underlying asset's price changes, is used for linear approximation.
The adjusted portfolio values are then used to calculate VaR and ES. This method assumes a linear relationship between the price changes of the underlying asset and the option prices.
The Delta-Normal approach simplifies the calculation and provides a view of risk under the assumption of linear price sensitivity and normal distribution of returns.

# Problem 3

Use the Fama French 3 factor return time series (F-F_Research_Data_Factors_daily.CSV) as well as the Carhart Momentum time series (F-F_Momentum_Factor_daily.CSV) to fit a 4 factor model to the following stocks.

AAPL FB UNH MA
MSFT NVDA HD PFE
AMZN BRK-B PG XOM
TSLA JPM V DIS
GOOGL JNJ BAC CSCO

Fama stores values as percentages, you will need to divide by 100 (or multiply the stock returns by 100) to get like units.

Based on the past 10 years of factor returns, find the expected annual return of each stock.

Construct an annual covariance matrix for the 10 stocks.

Assume the risk free rate is 0.0425. Find the super efficient portfolio.


In [32]:
import statsmodels.api as sm

ff = pd.read_csv('F-F_Research_Data_Factors_daily.csv', parse_dates=['Date']).set_index('Date')
mom = pd.read_csv('F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date').rename(columns={'Mom   ':  "Mom"})
factor = (ff.join(mom, how='right') / 100).loc['2013-1-31':]

all_prices = pd.read_csv('DailyPrices.csv', parse_dates=['Date'])
all_returns = pd.DataFrame(mf.return_calculate(all_prices)).set_index('Date')

/Users/yanghaochen/opt/anaconda3/lib/python3.9/site-packages/lib_hcy_week5/myfunctions.py:243: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars[i]] = p2[:,i]


In [33]:
stocks = ['AAPL', 'META', 'UNH', 'MA',
          'MSFT' ,'NVDA', 'HD', 'PFE',
          'AMZN' ,'BRK-B', 'PG', 'XOM',
          'TSLA' ,'JPM' ,'V', 'DIS',
          'GOOGL', 'JNJ', 'BAC', 'CSCO']

dataset = all_returns[stocks].join(factor)
reg_set = dataset.dropna()

In [34]:
reg_set.head()

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,...,DIS,GOOGL,JNJ,BAC,CSCO,Mkt-RF,SMB,HML,RF,Mom
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-15,0.023152,0.015158,0.008073,0.019724,0.018542,0.091812,0.004836,-0.000201,0.008658,0.006109,...,0.025655,0.007987,0.010326,0.007803,0.020496,0.0187,0.0133,-0.0142,0.0,-0.0091
2022-02-16,-0.001389,-0.020181,0.003806,0.003643,-0.001167,0.000604,-0.008974,-0.002209,0.010159,-0.001739,...,0.010535,0.008268,-0.000598,-0.002302,-0.000369,-0.0002,-0.0009,0.0031,0.0,0.0064
2022-02-17,-0.021269,-0.040778,-0.020227,-0.024103,-0.029282,-0.075591,-0.006141,-0.015700,-0.021809,-0.006653,...,-0.021746,-0.037746,-0.006100,-0.033767,0.028018,-0.0228,-0.0028,0.0110,0.0,0.0103
2022-02-18,-0.009356,-0.007462,-0.005379,-0.010035,-0.009631,-0.035296,-0.003075,-0.007566,-0.013262,0.003987,...,-0.010396,-0.016116,-0.010719,-0.002388,0.025820,-0.0087,-0.0009,0.0093,0.0,0.0104
2022-02-22,-0.017812,-0.019790,-0.011329,-0.004487,-0.000729,-0.010659,-0.088506,-0.020606,-0.015753,-0.002033,...,-0.021604,-0.004521,-0.013590,-0.008703,-0.015906,-0.0118,-0.0048,0.0011,0.0,0.0043


In [35]:
factors = ['Mkt-RF', 'SMB', 'HML', 'Mom']
X = reg_set[factors]
X = sm.add_constant(X)

y = reg_set[stocks].sub(reg_set['RF'], axis=0)
betas = pd.DataFrame(index=stocks, columns=factors)
alphas = pd.DataFrame(index=stocks, columns=['Alpha'])

In [36]:
for stock in stocks:
    ols_model = sm.OLS(y[stock], X).fit()
    betas.loc[stock] = ols_model.params[factors]
    alphas.loc[stock] = ols_model.params['const']

beta_return = pd.DataFrame(np.dot(factor[factors],betas.T), index=factor.index, columns=betas.index)
# The result of this matrix multiplication is a DataFrame where each row represents a date and each column represents a stock. The values in this DataFrame are the expected returns for each stock on each date, calculated based on the factor model.

beta_rf_return = pd.merge(beta_return,factor['RF'], left_index=True, right_index=True) # merge rows which beta_return and factor both have
alpha_beta_returns = beta_rf_return.add(beta_rf_return['RF'],axis=0).drop('RF',axis=1).add(alphas.T.loc['Alpha'], axis=1)
# Adding the Risk-Free Rate (RF) to the Returns of each column, Dropping the RF Column, Adding Alpha Values
# basically it is recalculating the returns for each stock at each time period using the factor regression formula and results - predicted stock returns

In [37]:
beta_rf_return.head()

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,...,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO,RF
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-31,-0.005799,-0.001919,-0.00606,-0.001668,-0.007596,0.000147,0.000475,-0.004713,-0.004641,-0.001071,...,0.004296,-0.002163,-0.00081,-0.002792,0.002782,-0.005099,-0.004655,-0.000418,-0.003013,0.0
2013-02-01,0.010949,0.008183,0.008843,0.010664,0.009485,0.016538,0.007911,0.007117,0.009454,0.009402,...,0.014264,0.010136,0.010236,0.008958,0.009072,0.008505,0.003911,0.011551,0.010143,0.0
2013-02-04,-0.012957,-0.012582,-0.008545,-0.012741,-0.011472,-0.021178,-0.009935,-0.007461,-0.013425,-0.01113,...,-0.015252,-0.014001,-0.012649,-0.010725,-0.012576,-0.01136,-0.003933,-0.014264,-0.011031,0.0
2013-02-05,0.012941,0.011363,0.009306,0.011717,0.012093,0.018344,0.008572,0.007987,0.012934,0.01029,...,0.012685,0.012547,0.011609,0.01024,0.010298,0.011289,0.004806,0.012959,0.010625,0.0
2013-02-06,-0.00067,0.002336,-0.002198,0.001372,-0.001509,0.003269,0.001841,-0.001071,0.001189,0.00187,...,0.004167,0.001259,0.002791,0.00062,0.004209,0.000176,-0.001553,0.003347,-0.000062,0.0


In [38]:
alpha_beta_returns.head()

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Date,,,,,,,,,,,,,,,,,,,,
2013-01-31,-0.005739,-0.002302,-0.005472,-0.001276,-0.00751,0.000551,0.000566,-0.004766,-0.005325,-0.000934,-0.005049,0.005973,-0.002853,-0.000783,-0.00228,0.001803,-0.005652,-0.0044,-0.001245,-0.002897
2013-02-01,0.01101,0.0078,0.009431,0.011056,0.009572,0.016943,0.008002,0.007064,0.00877,0.009539,0.006131,0.015941,0.009445,0.010263,0.00947,0.008093,0.007953,0.004166,0.010723,0.01026
2013-02-04,-0.012896,-0.012966,-0.007957,-0.012348,-0.011385,-0.020774,-0.009844,-0.007514,-0.014108,-0.010994,-0.006257,-0.013575,-0.014691,-0.012622,-0.010212,-0.013555,-0.011913,-0.003678,-0.015092,-0.010914
2013-02-05,0.013001,0.01098,0.009894,0.012109,0.01218,0.018749,0.008663,0.007933,0.01225,0.010426,0.006937,0.014362,0.011856,0.011636,0.010752,0.009319,0.010737,0.005061,0.012132,0.010742
2013-02-06,-0.000609,0.001953,-0.00161,0.001765,-0.001422,0.003673,0.001932,-0.001124,0.000506,0.002006,-0.001648,0.005844,0.000568,0.002818,0.001132,0.00323,-0.000376,-0.001299,0.002519,0.000055


In [39]:
beta_return.head()

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Date,,,,,,,,,,,,,,,,,,,,
2013-01-31,-0.005799,-0.001919,-0.00606,-0.001668,-0.007596,0.000147,0.000475,-0.004713,-0.004641,-0.001071,-0.005039,0.004296,-0.002163,-0.00081,-0.002792,0.002782,-0.005099,-0.004655,-0.000418,-0.003013
2013-02-01,0.010949,0.008183,0.008843,0.010664,0.009485,0.016538,0.007911,0.007117,0.009454,0.009402,0.00614,0.014264,0.010136,0.010236,0.008958,0.009072,0.008505,0.003911,0.011551,0.010143
2013-02-04,-0.012957,-0.012582,-0.008545,-0.012741,-0.011472,-0.021178,-0.009935,-0.007461,-0.013425,-0.01113,-0.006247,-0.015252,-0.014001,-0.012649,-0.010725,-0.012576,-0.01136,-0.003933,-0.014264,-0.011031
2013-02-05,0.012941,0.011363,0.009306,0.011717,0.012093,0.018344,0.008572,0.007987,0.012934,0.01029,0.006946,0.012685,0.012547,0.011609,0.01024,0.010298,0.011289,0.004806,0.012959,0.010625
2013-02-06,-0.00067,0.002336,-0.002198,0.001372,-0.001509,0.003269,0.001841,-0.001071,0.001189,0.00187,-0.001639,0.004167,0.001259,0.002791,0.00062,0.004209,0.000176,-0.001553,0.003347,-0.000062


In [40]:
expected_annual_return = ((alpha_beta_returns+1).cumprod().tail(1) ** (1/alpha_beta_returns.shape[0]) - 1) * 252
# .cumprod(): This calculates the cumulative product of the adjusted returns. The cumulative product is used to simulate the compounding effect of the returns over time.
# .tail(1): This selects the last row of the cumulative product, which is the very last compounding return
expected_annual_return = expected_annual_return.reset_index()
# If the DataFrame's index is currently set to dates or any other specific identifier, reset_index() will move this index to a regular column and create a new, default integer index
expected_annual_return = expected_annual_return.melt(id_vars=['Date'], var_name='Stock', value_name='Annual Return')
# The melt function is used to transform the DataFrame from a wide format to a long format.
# id_vars=['Date']: This specifies that the 'Date' column (or columns specified in id_vars) should remain as is, and not be part of the reshaping
# var_name='Stock': This sets the name of the new column that will contain what were previously the column headers (stock names, in this case) in the wide format.
# value_name='Annual Return': This sets the name for the new column that will contain the values corresponding to each stock.
# 至于为什么要melt把wide变成long，这可以方便处理nested data，具体如最下方展示
expected_annual_return.drop('Date', axis=1, inplace=True)
print(expected_annual_return)

    Stock Annual Return
0    AAPL      0.157144
1    META      0.017941
2     UNH        0.2538
3      MA      0.222901
4    MSFT      0.155944
5    NVDA      0.279721
6      HD      0.120591
7     PFE      0.076962
8    AMZN     -0.042945
9   BRK-B      0.129923
10     PG       0.08154
11    XOM      0.521821
12   TSLA     -0.033253
13    JPM      0.098273
14      V      0.241054
15    DIS     -0.155372
16  GOOGL     -0.017075
17    JNJ      0.124206
18    BAC     -0.112301
19   CSCO      0.147807


In [41]:
cov_mtx = all_returns[stocks].cov()*252
print(cov_mtx)

           AAPL      META       UNH        MA      MSFT      NVDA        HD  \
AAPL   0.126877  0.139557  0.037447  0.081272  0.102937  0.171265  0.066193   
META   0.139557  0.400843  0.017102  0.102465  0.142255  0.240599  0.098845   
UNH    0.037447  0.017102  0.060922  0.031117  0.036318  0.046531  0.026045   
MA     0.081272  0.102465  0.031117  0.095762  0.079856  0.137369  0.056792   
MSFT   0.102937  0.142255  0.036318  0.079856  0.127839  0.175956  0.070916   
NVDA   0.171265  0.240599  0.046531  0.137369  0.175956  0.403814  0.112055   
HD     0.066193  0.098845  0.026045  0.056792  0.070916  0.112055  0.097074   
PFE    0.032745  0.045091  0.032068  0.033440  0.035082  0.046362  0.033271   
AMZN   0.122117  0.194794  0.034737  0.096194  0.133856  0.221364  0.096833   
BRK-B  0.055520  0.061619  0.028173  0.047520  0.052676  0.084290  0.042310   
PG     0.036945  0.033637  0.027861  0.031163  0.033859  0.041944  0.034414   
XOM    0.037700  0.020759  0.026843  0.030837  0.031

In [42]:
def super_efficient_portfolio(returns, rf_rate, cov_matrix): # all has to be annualized
    annual_returns = returns['Annual Return'].values.T
    num_stocks = len(returns)

    def neg_sharpe_ratio(weights):
        pfl_return = np.sum(annual_returns * weights)
        pfl_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        sharpe_ratio = (pfl_return - rf_rate) / pfl_std_dev
        return -sharpe_ratio

    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
                   {'type': 'ineq', 'fun': lambda w: w}]

    bounds = [(0, 1) for _ in range(num_stocks)]

    init_weights = np.ones(num_stocks) / num_stocks
    opt_result = scipy.optimize.minimize(neg_sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)

    # Return optimal weights and Sharpe ratio of resulting portfolio
    super_weights = opt_result.x
    super_sharpe_ratio = - neg_sharpe_ratio(super_weights)

    return super_weights*100, super_sharpe_ratio

In [43]:
weights, sharpe_ratio = super_efficient_portfolio(expected_annual_return, 0.0425, cov_mtx)

weights_df = pd.DataFrame(columns=['Stock', 'Weight'])
weights_df['Stock'] = stocks
weights_df['Weight'] = weights.round(2)
print(weights_df)
print(f"Sharpe Ratio of super efficient portfolio is {sharpe_ratio:.2f}")

    Stock  Weight
0    AAPL    0.00
1    META    0.00
2     UNH   22.57
3      MA    0.00
4    MSFT    0.00
5    NVDA    0.00
6      HD    0.00
7     PFE    0.00
8    AMZN    0.00
9   BRK-B    0.00
10     PG    0.00
11    XOM   57.44
12   TSLA    0.00
13    JPM    0.00
14      V   12.93
15    DIS    0.00
16  GOOGL    0.00
17    JNJ    7.05
18    BAC    0.00
19   CSCO    0.00
Sharpe Ratio of super efficient portfolio is 1.47


In [44]:
betas.head()

,Mkt-RF,SMB,HML,Mom
AAPL,1.215619,-0.520657,-0.458912,0.122752
META,1.15182,-0.458172,-0.283978,-0.715352
UNH,0.805859,-0.396701,-0.287857,0.591512
MA,1.078447,-0.143987,0.064123,-0.021296
MSFT,1.142627,-0.793738,-0.596419,0.051736


In [52]:
test1 = ((alpha_beta_returns+1).cumprod().tail(1) ** (1/alpha_beta_returns.shape[0]) - 1) * 252
test1.head()

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Date,,,,,,,,,,,,,,,,,,,,
2023-01-31,0.157144,0.017941,0.2538,0.222901,0.155944,0.279721,0.120591,0.076962,-0.042945,0.129923,0.08154,0.521821,-0.033253,0.098273,0.241054,-0.155372,-0.017075,0.124206,-0.112301,0.147807


In [54]:
test = test1.reset_index()
test = test.melt(var_name='Stock', value_name='Annual Return')
test.drop(index=0, inplace=True)
test.head()

,Stock,Annual Return
1,AAPL,0.157144
2,META,0.017941
3,UNH,0.2538
4,MA,0.222901
5,MSFT,0.155944


super_efficient_portfolio()

Inputs:
returns: A DataFrame containing the expected annual returns of each stock.
rf_rate: The risk-free rate, used in the calculation of the Sharpe Ratio.
cov_matrix: The covariance matrix of the stock returns, used to calculate portfolio risk.

Annual Returns Preparation:
annual_returns = returns['Annual Return'].values.T: Extracts the annual returns from the DataFrame and transposes them into a format suitable for calculations.

Number of Stocks:
num_stocks = len(returns): Determines the number of stocks in the portfolio.

Negative Sharpe Ratio Function:
neg_sharpe_ratio(weights): A function that calculates the negative Sharpe Ratio of a portfolio with given weights. This is used because optimization functions typically minimize a value, and we want to maximize the Sharpe Ratio.
Inside this function:
pfl_return = np.sum(annual_returns * weights): Calculates the expected portfolio return based on the weights and individual stock returns.
pfl_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))): Calculates the standard deviation (risk) of the portfolio, which is a measure of volatility.
The Sharpe Ratio is then calculated and returned as a negative value for minimization.

Constraints and Bounds:
constraints: Defines the constraints for the optimization:
The sum of weights must equal 1 ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1}).
Each weight must be non-negative ({'type': 'ineq', 'fun': lambda w: w}).
bounds: Sets the bounds for each weight to be between 0 and 1 (inclusive).

Optimization Process:
init_weights = np.ones(num_stocks) / num_stocks: Initializes the weights to be equal for all stocks.
opt_result = scipy.optimize.minimize(...): Uses the minimize function from scipy.optimize to find the set of weights that minimize the negative Sharpe Ratio (thereby maximizing the actual Sharpe Ratio).

Output:
super_weights = opt_result.x: Extracts the optimal weights from the optimization result.
super_sharpe_ratio = - neg_sharpe_ratio(super_weights): Calculates the Sharpe Ratio of the optimal portfolio.
The function returns the optimal weights multiplied by 100 (to convert to percentages) and the Sharpe Ratio of the portfolio.

Data Import and Preparation:
The Fama French 3 factor and Carhart Momentum factor datasets are imported from CSV files and merged. The values are divided by 100 to convert them from percentages to decimal format.
Daily stock prices are also imported, and returns are calculated using a function VaR.return_calculate.
A list of specific stocks of interest is defined.
The returns data for these stocks is merged with the factor data to create a combined dataset.

Dataset Preparation for Regression:
The combined dataset is cleaned of any missing values (NaNs).
The factors (Mkt-RF, SMB, HML, Mom) and stock returns are isolated for regression analysis.
The risk-free rate is subtracted from the stock returns.

Regression Analysis:
A loop runs a regression analysis for each stock using the OLS (Ordinary Least Squares) method.
It calculates the beta values for each stock with respect to the factors and stores them.
It also calculates the alpha (constant term) for each stock.

Calculation of Expected Returns:
The beta values are used to calculate the expected returns based on the factor model.
These expected returns are adjusted for the risk-free rate and then added to the alpha values.
The annual expected returns are then calculated.

Annual Covariance Matrix:
The annual covariance matrix of the stock returns is calculated, a crucial step for portfolio optimization.

Portfolio Optimization - Finding the 'Super Efficient Portfolio':
A function super_efficient_portfolio is defined for this purpose.
It uses the Sharpe Ratio as the optimization criterion, aiming to maximize this ratio.
The function uses scipy's minimize function with constraints that ensure the sum of weights is 1 and all weights are non-negative.
The optimal weights for each stock in the portfolio that maximize the Sharpe Ratio are computed.

Output:
The weights of the stocks in the 'super efficient' portfolio and the Sharpe Ratio of this portfolio are printed out.